In [1]:

%matplotlib inline
from itertools import chain

from cmdstanpy import CmdStanModel
import numpy as np
import pandas as pd
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
from random import choices



/home/josh/workspace/bayesianquilts/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_probability.python.bijectors as tfb

from bayesianquilts.models.logistic_regression_reparam import LogisticRegression2
from bayesianquilts.metrics.classification import classification_metrics
from bayesianquilts.sampler import psis, nppsis

In [6]:
from ipykernel import get_connection_file
import os

connection_file = get_connection_file()
notebook_path = os.path.dirname(connection_file)
print(notebook_path)

/home/josh/.local/share/jupyter/runtime


In [8]:
roach = pd.read_csv(Path.home() / "workspace" / "bayesianquilts"/ "notebooks"/ "roach" / "roachdata.csv")
roach

,Unnamed: 0,y,roach1,treatment,senior,exposure2
0,1,153,308.00,1,0,0.800000
1,2,127,331.25,1,0,0.600000
2,3,7,1.67,1,0,1.000000
3,4,7,3.00,1,0,1.000000
4,5,0,2.00,1,0,1.142857
...,...,...,...,...,...,...
257,258,0,0.00,0,1,1.485714
258,259,0,0.00,0,1,1.000000
259,260,0,0.00,0,1,1.000000
260,261,171,0.00,0,1,1.000000


In [9]:

roach = pd.read_csv(Path.home() / "workspace" / "bayesianquilts"/ "notebooks"/ "roach" / "roachdata.csv")
y_ = (roach['y']>0).astype(int)
X_ = roach.iloc[:, 2:]
X_scaled = (X_ - X_.mean())/X_.std()
X_scaled = X_scaled.fillna(0).to_numpy()
y_ = y_.to_numpy()
N = X_scaled.shape[0]
d = X_scaled.shape[1]

print((N, d))

(262, 4)


In [10]:
sm = CmdStanModel(stan_file="notebooks/ovarian/ovarian_model.stan")

In [12]:
guessnumrelevcov = 20 # 20.
slab_scale = 2.5/100
scale_icept = 5.0
nu_global = 1
nu_local = 1
slab_df = 1
scale_global = guessnumrelevcov / ((d - guessnumrelevcov) * np.sqrt(N))/10000

control = {"adapt_delta": 0.9995, "max_treedepth": 15}

roach_data = {
    "N": N,
    "d": d,
    "slab_df": slab_df,
    "slab_scale": slab_scale,
    "scale_icept": scale_icept,
    "nu_global": 1,
    "nu_local": 1,
    "scale_global": scale_global,
    "y": y_.astype(int)[:].tolist(),
    "x": X_scaled.tolist(),
}

import json
with open("notebooks/roach/roach_data.json", 'w') as f:
    json.dump(roach_data, f)

In [14]:
fit = sm.sample(
    data="notebooks/roach/roach_data.json",
    iter_warmup=20000,
    iter_sampling=2000,
    thin=2,
    adapt_delta=0.9995,
    max_treedepth=15,
)

02:49:06 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status


02:49:06 - cmdstanpy - ERROR - Chain [2] error: error during processing Operation not permitted
02:49:06 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
02:49:06 - cmdstanpy - ERROR - Chain [3] error: error during processing Operation not permitted
02:49:06 - cmdstanpy - ERROR - Chain [4] error: error during processing Operation not permitted
chain 1 |██████████| 00:00 Sampling completed
chain 2 |██████████| 00:00 Sampling completed
chain 3 |██████████| 00:00 Sampling completed
chain 4 |██████████| 00:00 Sampling completed


02:49:06 - cmdstanpy - INFO - CmdStan done processing.


RuntimeError: Error during sampling:
Exception: ovarian_model_model_namespace::ovarian_model_model: scale_global is -7.72253e-06, but must be greater than or equal to 0.000000 (in 'ovarian_model.stan', line 8, column 2 to column 31)
Exception: ovarian_model_model_namespace::ovarian_model_model: scale_global is -7.72253e-06, but must be greater than or equal to 0.000000 (in 'ovarian_model.stan', line 8, column 2 to column 31)
Exception: ovarian_model_model_namespace::ovarian_model_model: scale_global is -7.72253e-06, but must be greater than or equal to 0.000000 (in 'ovarian_model.stan', line 8, column 2 to column 31)
Exception: ovarian_model_model_namespace::ovarian_model_model: scale_global is -7.72253e-06, but must be greater than or equal to 0.000000 (in 'ovarian_model.stan', line 8, column 2 to column 31)
Command and output files:
RunSet: chains=4, chain_ids=[1, 2, 3, 4], num_processes=4
 cmd (chain 1):
	['/home/josh/workspace/bayesianquilts/notebooks/ovarian/ovarian_model', 'id=1'